In [2]:
import pandas as pd
import numpy as np

In [3]:
data_all = pd.read_csv('vars 308.csv')

In [4]:
data_all.head()

,record,fraud_label,dow_risk,ssn_day_since,ssn_count_0,ssn_count_1,ssn_count_3,ssn_count_7,ssn_count_14,ssn_count_30,...,ssn_homephone_count_0_by_3,ssn_homephone_count_0_by_7,ssn_homephone_count_0_by_14,ssn_homephone_count_0_by_30,ssn_homephone_count_1_by_3,ssn_homephone_count_1_by_7,ssn_homephone_count_1_by_14,ssn_homephone_count_1_by_30,ssn_name_count_0_by_3,ssn_name_count_0_by_7
0,1,0,0.014499,0.0,1,1,1,1,1,1,...,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0
1,2,1,0.014499,0.0,1,1,1,1,1,1,...,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0
2,3,0,0.014499,0.0,1,1,1,1,1,1,...,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0
3,4,0,0.014499,0.0,1,1,1,1,1,1,...,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0
4,5,0,0.014499,0.0,1,1,1,1,1,1,...,3.0,7.0,14.0,30.0,3.0,7.0,14.0,30.0,3.0,7.0


In [5]:
data = data_all[data_all['record'] <= 833508]
data = data[data['record'] >= 35755]
data.drop(['record'],axis=1,inplace=True)
data['RANDOM'] = np.random.ranf(len(data))

## Filter

In [6]:
num_vars = 311
goods = data[data['fraud_label'] == 0]
bads = data[data['fraud_label'] == 1]

KSFDR = pd.DataFrame(np.zeros((num_vars+1,3)))
i = 0
for col in data:
        KSFDR.loc[i,0] = col
        i = i+1
KSFDR.columns=['feature','KS','FDR']

In [7]:
num_bads = bads.shape[0]

In [8]:
from scipy.stats import ks_2samp
from sklearn.feature_selection import SelectKBest

In [ ]:
j = 0
for col in data:
    KSFDR['KS'][j] = ks_2samp(goods[col],bads[col])[0]
    j = j+1
KSFDR.sort_values(by=['KS'], ascending = False, inplace = True)
KSFDR.head()

c:\users\ss\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
cut = int(round(len(data)*0.03))
k = 0
for col in data:
    x1 = data.sort_values(col, ascending = False).head(cut)
    x2 = data.sort_values(col, ascending = True).head(cut)
    f1 = x1.loc[:,'fraud_label']
    f2 = x2.loc[:,'fraud_label']
    FDR1 = sum(f1)/num_bads
    FDR2 = sum(f2)/num_bads
    FDR = max(FDR1,FDR2)
    KSFDR.loc[k,'FDR'] = FDR
    k = k+1

In [ ]:
KSFDR['rank_ks'] = KSFDR['KS'].rank(ascending = True)
KSFDR['rank_FDR'] = KSFDR['FDR'].rank(ascending = True)
KSFDR['Avg_rank'] = (KSFDR['rank_ks']+KSFDR['rank_FDR'])/2
KSFDR.sort_values(by=['avg_rank'], ascending = False, inplace = True)

In [ ]:
KSFDR.reset_index(drop=True)
num_keep = 70
cols_keep = list(KSFDR['field'][1:num_keep])
cols_keep

## Wrapper

In [ ]:
newX = data.drop(['fraud_label'],axis=1,inplace=True).loc[:,cols_keep]
y = data.fraud_label

In [ ]:
# wrapper
import sklearn
from sklearn.feature_selection import RFECV
from sklearn.linear_model import LogisticRegression

# step: If greater than or equal to 1, 
# then step corresponds to the (integer) number of features to remove at each iteration. 
# If within (0.0, 1.0), then step corresponds to the percentage (rounded down) of 
# features to remove at each iteration

# cv: cross-validation generator or an iterable

# Verbose: Controls verbosity of output

# n_jobs: Number of cores to run in parallel while fitting across folds

model = LogisticRegression()
refec = RFECV(estimator = model, step=1, cv=3, verbose=3, n_jobs=-1, scoring = 'roc_auc')
refec.fit(newX, y)

In [ ]:
# x - Number of Features
# y - Performance Score (ROC_AUC)
import matplotlib.pyplot as plt
plt.figure()
plt.plot(range(1,len(refec.grid_scores_)+1), refec.grid_scores_)